In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
def scheduleDataScrape(htmlID, driver):
    
    #Finds the correct week button and clicks it
    element = driver.find_element(By.ID, "{}".format(htmlID))
    element.click()
    time.sleep(5)

    #Finds the table using BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': 'table table-sm table-bordered table-striped table--statistics'})

    #Creates an empty dataframe then copies website table to it
    dataframe = pd.DataFrame()
    headers = [header.text.strip() for header in table.find_all('th')]
    for row in table.find_all('tr'):
        data = [cell.text.strip() for cell in row.find_all('td')]
        if len(data) == len(headers):
            dataframe = dataframe.append(pd.Series(data, index=headers), ignore_index=True)
    
    #Returns the copied dataframe
    return dataframe



We'll now pass in the html button IDs to the function and run the webdriver 

In [ ]:
#Setting file path of webdriver and open website
PATH = "C:\FILE PATH OF YOUR WEBDRIVER\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://hashtagbasketball.com/advanced-nba-schedule-grid")

#Assigns the correct html ID to each button
week19ID = "ContentPlaceHolder1_Button19Y"
week20ID = "ContentPlaceHolder1_Button20Y"
week21ID = "ContentPlaceHolder1_Button21Y"

#Calls the function to click and load data into dataframe
df1 = scheduleDataScrape(week19ID, driver)
df2 = scheduleDataScrape(week20ID, driver)
df3 = scheduleDataScrape(week21ID, driver)

#Closes webdriver
driver.close()

Check if the dataframes were loaded correctly

In [ ]:
df1.info()

In [ ]:
df1.head(30)

The dataframes have some inconsistent rows run these next to see if the 'Games' column still contains non-integer values

In [ ]:
df1.index[df1['Games'] == "Games"].tolist()

In [ ]:
df2.index[df2['Games'] == "Games"].tolist()

In [ ]:
df3.index[df3['Games'] == "Games" ].tolist()

Drop the unnecessary rows from the dataframes then reindex the dataframe

In [ ]:
df1 = df1.drop(df1.index[df1['Games'] == "Games"].tolist())
df1 = df1.drop(0)
df1 = df1.reset_index(drop=True)
df2 = df2.drop(df2.index[df2['Games'] == "Games"].tolist())
df2 = df2.drop(0)
df2 = df2.reset_index(drop=True)
df3 = df3.drop(0)
df3 = df3.reset_index(drop=True)
df1.head(30)

In [ ]:
df1['Games'] = df1['Games'].astype(int)
df2['Games'] = df2['Games'].astype(int)
df3['Games'] = df3['Games'].astype(int)

Create new dataframes containing only weekly game count

In [ ]:
new_df1 = pd.DataFrame({'Team': df1['Team'], 'Games1': df1['Games']})
new_df2 = pd.DataFrame({'Games2': df2['Games']})
new_df3 = pd.DataFrame({'Games3': df3['Games']})
merged_df = pd.merge(new_df1, new_df2, left_index=True, right_index=True)
merged_df = pd.merge(merged_df, new_df3, left_index=True, right_index=True)
merged_df["Total Games"] = merged_df.sum(axis=1)
merged_df.head(30)

In [ ]:
#Setting file path of webdriver and open website
PATH = "C:\FILE PATH OF YOUR WEBDRIVER\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://hashtagbasketball.com/fantasy-basketball-rankings")

#Finds the drop down element for top players and selects 'Top 300'
find_Top_300 = driver.find_element(By.ID, "ContentPlaceHolder1_DDSHOW")
select_Top_300 = Select(find_Top_300)
select_Top_300.select_by_visible_text("300")
time.sleep(5)

#Finds the drop down element for collection of gamelogs and selects 'Last 30 Days'
find_Range = driver.find_element(By.ID, "ContentPlaceHolder1_DDDURATION")
select_Range = Select(find_Range)
select_Range.select_by_value("30")
time.sleep(5)

#Finds the table using BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', {'class': 'table table-sm table-bordered table-striped table--statistics'})

#Creates an empty dataframe then copies website table to it
playerDf = pd.DataFrame()
headers = [header.text.strip() for header in table.find_all('th')]
for row in table.find_all('tr'):
    data = [cell.text.strip() for cell in row.find_all('td')]
    if len(data) == len(headers):
        playerDf = playerDf.append(pd.Series(data, index=headers), ignore_index=True)

#Closes the webdriver
driver.quit()
playerDf.head(50)

In [ ]:
playerDf.index[playerDf['R#'] == "R#"].tolist()

In [ ]:
playerDf = playerDf.drop(playerDf.index[playerDf['R#'] == "R#"].tolist())
playerDf = playerDf.reset_index(drop=True)
playerDf = playerDf.drop(['R#'], axis=1)

playerDf['GP'] = playerDf['GP'].astype(int)
playerDf = playerDf.astype({'MPG': 'float', '3PM' : 'float', 'PTS' : 'float', 'TREB' : 'float', 'AST' : 'float', 'STL' : 'float', 'BLK' : 'float', 'TO' : 'float'})
playerDf.info()

In [ ]:
playerDf[['FG%', 'FG Volume']] = playerDf['FG%'].str.split('\n\n', expand=True)
playerDf[['FT%', 'FT Volume']] = playerDf['FT%'].str.split('\n\n', expand=True)
playerDf['FG%'] = playerDf['FG%'].astype(float)
playerDf['FT%'] = playerDf['FT%'].astype(float)
playerDf['FG Volume'] = playerDf['FG Volume'].astype(str)
playerDf['FT Volume'] = playerDf['FT Volume'].astype(str)
playerDf.head()

Expand the FG% and FT% Volume columns to obtain makes and attempts columns

In [ ]:
playerDf[['FG Volume (Makes)', 'FG Volume (Attempts)']] = playerDf['FG Volume'].str.extract(r'\((\d+\.\d+)\/(\d+\.\d+)\)')
playerDf[['FT Volume (Makes)', 'FT Volume (Attempts)']] = playerDf['FT Volume'].str.extract(r'\((\d+\.\d+)\/(\d+\.\d+)\)')
playerDf.tail(50)

Let's check the previous dataframe we made and find any identity keys we can join with

In [ ]:
merged_df.head(40)

There aren't any identity keys that match our datasets so we'll have to create a new column to the merged_df with team name abbreviations that match the playerDf

In [ ]:
merged_df['TEAM'] = ['ATL', 'BOS', 'BRO', 'CHA', 'CHI' , 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKL', 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
merged_df.head(30)

Merge both dataframes to create final dataframe

In [ ]:
finalDf = pd.merge(playerDf, merged_df, on='TEAM')
finalDf.head(30)

In [ ]:
finalDf.info()

Now export the final dataframe as a csv

In [ ]:
finalDf.to_csv('FantasyPlayoffsTop300.csv', index=False)